In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

In [2]:
if 'google.colab' in str(get_ipython()): # running in colab
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = '/content/drive/MyDrive/revisiting_crossval_RNAfolding/'
else:
    DATA_PATH = '/home/lbugnon/shared_insync/revisiting_crossval_RNAfolding/'

In [3]:
!pip install git+https://github.com/lbugnon/sincfold_utils 

  Cloning https://github.com/lbugnon/sincfold_utils to /tmp/pip-req-build-z9egk5f9
  Running command git clone --filter=blob:none --quiet https://github.com/lbugnon/sincfold_utils /tmp/pip-req-build-z9egk5f9
  Resolved https://github.com/lbugnon/sincfold_utils to commit f9fa361eae7b90cc36fc9fc776a37dafb2279571
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
dataset = pd.read_csv(f"{DATA_PATH}data/archiveII_220808.csv", index_col="id")
dataset.head()

,Unnamed: 0,method,sequence,structure,motif,base_pairs,header,date,resolution,len
id,,,,,,,,,,
5s_Acholeplasma-laidlawii-1,0,NaN,UCUGGUGACGAUAGGUAAGAUGGUUCACCUGUUCCCAUCCCGAACA...,((((((((......((((((((....((((((.............)...,NaN,"[[1, 111], [2, 110], [3, 109], [4, 108], [5, 1...",NaN,NaN,NaN,112
5s_Acidovorax-temperans-1,1,NaN,UGCCUGAUGACCAUAGCAAGUUGGUACCACUCCUUCCCAUCCCGAA...,.(((((((((.....((((((((.....((((((...............,NaN,"[[2, 115], [3, 114], [4, 113], [5, 112], [6, 1...",NaN,NaN,NaN,115
tmRNA_Stre.gord._TRW-29390_1-349,2,NaN,GGGGUCGUUACGGAUUCGACAGGCAUUAUGAGGCAUAUUUUGCGAC...,(((((((............((((((((....(((((((((..((((...,NaN,"[[1, 345], [2, 344], [3, 343], [4, 342], [5, 3...",NaN,NaN,NaN,349
tRNA_tdbR00000055-Schizosaccharomyces_pombe-4896-Glu-3UC,3,NaN,UCCGUUGUGGUCCAACGGCUAGGAUUCGUCGCUUUCACCGACGGGA...,(((((((..((((........))))((((((.......)))))).....,NaN,"[[1, 71], [2, 70], [3, 69], [4, 68], [5, 67], ...",NaN,NaN,NaN,75
srp_List.mono._U15684,4,NaN,UGGGUUGAUGAGCGUGAAGCCUUCGCUCGGUUGGAUUUUUCUUCAU...,.(.((((...(.(.((.(.((..(.....)..)).)...(...(.....,NaN,"[[2, 276], [4, 274], [5, 273], [6, 272], [7, 2...",NaN,NaN,NaN,279


## Random k-fold

In [8]:
splits = pd.read_csv(f"{DATA_PATH}data/1_random_kfolds/ArchiveII_splits_random.csv")
splits.head()

,fold_number,partition,id
0,0,train,tmRNA_Stre.gord._TRW-29390_1-349
1,0,train,tRNA_tdbR00000055-Schizosaccharomyces_pombe-48...
2,0,train,srp_List.mono._U15684
3,0,train,srp_Vibr.fisc._CP000020
4,0,train,srp_Baci.thur._D11412
